In [ ]:
import json
import requests
import itertools
from collections import Counter, OrderedDict
import pandas
%matplotlib inline

In [ ]:
json_path = '/nfs/misc/literature/machine-learning/evaluation/2000articles/europePMC-NER/annotations_API/2k-annos.json'

In [ ]:
with open(json_path, 'r') as f:
    total_json = {}
    count = 0
    k_annos = []
    k_exacts = []
    k_preds = []
    k_missing = []
    for line in f:
        count += 1
        article = json.loads(line.strip())
        print('\n')
        print("========"*2+str(count)+"========"*2)
        print(f"article: {article['pmcid']}")
        anno_sent = []
        exacts = []
        pred_exact = []
   
        if article['annotations']:
            for anno in article['annotations']:
                anno_sent.append(anno['prefix'] + anno['exact'] + anno['postfix'])
                exacts.append(anno['exact'])
        k_annos.append(anno_sent)
        k_exacts.append(exacts)
              
              
        if anno_sent:
            predictions = requests.post('http://10.7.75.4:5201/pcse_ner_multi_batch_predictor',data={'text_sentence':json.dumps(anno_sent)}).json()
#             print(predictions['tagged'])  
            for pred_sent in predictions['tagged']:
                try:
                    pred_exact.append(pred_sent['exact'])
                except TypeError:
                    pass

        k_preds.append(pred_exact)
   
              
        if count > 2000:
            break
        if exacts or pred_exact:
            print('anno sent:')
            print(anno_sent)
            print('anno exact:')
            print(exacts)
            print('pred exact:')
            print(pred_exact)
 
              
#     if len(exacts) != len(pred_exact):
#         raise

In [ ]:
import pickle

path = '/nfs/gns/literature/machine-learning/Santosh/FP_analysis/'

with open(path+'k_exacts', 'rb') as fp:
    k_exacts = pickle.load(fp)
    
with open(path+'k_preds', 'rb') as fp:
    k_preds = pickle.load(fp) 
    
with open(path+'k_annos', 'rb') as fp:
    k_annos = pickle.load(fp)

In [ ]:
def no_intersection(lst1, lst2): 
    miss = []
    temp = set(lst2) 
    for value in lst1:
        scores = [fuzz.partial_ratio(value,sublist) for sublist in temp]
        if 100 not in scores:
            miss.append(value)  
    return miss
    

In [ ]:
from tqdm import tqdm
k_missing = []
for i,j in tqdm(zip(k_exacts,k_preds)):
    if i:
        missing = no_intersection(i, j)
        if missing:
            k_missing.append(missing)

In [ ]:
missing_data = list(itertools.chain.from_iterable(k_missing))
count = Counter(missing_data)
y = OrderedDict(count.most_common())

In [ ]:
count = Counter(missing_data)
y = OrderedDict(count.most_common())

In [ ]:
import csv
with open(path+"frequencies.csv", "w") as outfile:
    csvwriter = csv.writer(outfile)
    csvwriter.writerows(y.items())

In [ ]:
df = pandas.DataFrame.from_dict(y, orient='index')
df.plot(kind='bar')

In [ ]:
import pickle

path = '/nfs/gns/literature/machine-learning/Santosh/FP_analysis/2K/'
with open(path+'k_exacts', 'wb') as fp:
    pickle.dump(k_exacts, fp)
    
with open(path+'k_preds', 'wb') as fp:
    pickle.dump(k_preds, fp) 
    
with open(path+'k_annos', 'wb') as fp:
    pickle.dump(k_annos, fp)     
    

In [ ]:
# path = '/nfs/gns/literature/machine-learning/Santosh/FP_analysis/2K/'
# with open(path+'k_exacts', 'rb') as fp:
#     k_exacts = pickle.load(fp)
    
# with open(path+'k_preds', 'rb') as fp:
#     k_preds = pickle.load(fp) 
    
# k_exacts[6], k_preds[6]    

In [ ]:
anno_data = list(itertools.chain.from_iterable(k_exacts))
anno_count = Counter(anno_data)
anno_y = OrderedDict(anno_count.most_common())

In [ ]:
anno_y

In [ ]:
anno_data = list(itertools.chain.from_iterable(k_exacts))
anno_count = Counter(anno_data)
anno_y = OrderedDict(anno_count.most_common())

with open(path+"anno_frequencies.csv", "w") as outfile:
    csvwriter = csv.writer(outfile)
    anno_values = []
    for keys, values in y.items():
        csvwriter.writerows([str(anno_y[keys])])
    


# 20 K analysis. Move this to 20K file

In [41]:


import pandas as pd
from ast import literal_eval


file_20l_path = '/nfs/gns/literature/machine-learning/evaluation/20K-ML-NER/'
file_20k_name = '20K-GP_PCSE-Annotations.csv'


file_20k_df = pd.read_csv(file_20l_path+file_20k_name, names=['pmc_id', 'section', 'sentence', 'epmc_ner','ml_ner'],sep='\t')




In [42]:
file_20k_df.columns

Index(['pmc_id', 'section', 'sentence', 'epmc_ner', 'ml_ner'], dtype='object')

In [43]:
k_exacts = []
k_preds = []
k_missing = []



from tqdm import tqdm
for index_, each_annotation in tqdm(enumerate(file_20k_df.itertuples()),total=len(file_20k_df)):
#     if index_ <10:
    #     pmc_id = each_annotation.pmc_id
    #     section = each_annotation.section
    #     epmc_sentence = each_annotation.sentence
        k_exacts.append(literal_eval(each_annotation.epmc_ner))
        k_preds.append(literal_eval(each_annotation.ml_ner))

    

100%|██████████| 72972/72972 [00:01<00:00, 68681.35it/s]


In [44]:
k_exacts[1010], k_preds[1010]

(['SOM'], [])

In [45]:
def no_intersection(lst1, lst2): 
    miss = []
    temp = set(lst2) 
    for value in lst1:
        scores = [fuzz.partial_ratio(value,sublist) for sublist in temp]
        if 100 not in scores:
#         if any(i < 70for i in scores):
            miss.append(value)  
    return miss
    

In [46]:
import json
import requests
import itertools
from collections import Counter, OrderedDict
from fuzzywuzzy import fuzz


In [47]:
k_missing = []
for i,j in tqdm(zip(k_exacts,k_preds), total = len(k_exacts)):
    if i:
        missing = no_intersection(i, j)
        if missing:
            k_missing.append(missing)

100%|██████████| 72972/72972 [00:00<00:00, 80962.46it/s] 


In [48]:
missing_data = list(itertools.chain.from_iterable(k_missing))
count = Counter(missing_data)
y = OrderedDict(count.most_common())
y

OrderedDict([('Cer', 1236),
             ('ABA', 596),
             ('COI', 559),
             ('BPA', 522),
             ('PC1', 508),
             ('PHB', 487),
             ('Mim', 406),
             ('MIC', 383),
             ('SRI', 381),
             ('std', 364),
             ('PC2', 363),
             ('HGT', 362),
             ('GFP', 335),
             ('ssp', 335),
             ('FST', 331),
             ('NBS', 330),
             ('ESI', 323),
             ('ROI', 322),
             ('cav', 313),
             ('VT1', 303),
             ('CA1', 296),
             ('ECM', 288),
             ('LPS', 281),
             ('IND', 275),
             ('ppm', 273),
             ('TBA', 271),
             ('ANF', 255),
             ('CH2', 254),
             ('CNF', 251),
             ('NIR', 228),
             ('tri', 220),
             ('PLP', 219),
             ('NADH', 202),
             ('SPR', 201),
             ('Ln1', 197),
             ('PSII', 189),
             ('MS2', 183)

In [49]:
import pickle

path = '/nfs/gns/literature/machine-learning/Santosh/FP_analysis/20K/'
with open(path+'k_exacts', 'wb') as fp:
    pickle.dump(k_exacts, fp)
    
with open(path+'k_preds', 'wb') as fp:
    pickle.dump(k_preds, fp) 
    
# with open(path+'k_annos', 'wb') as fp:
#     pickle.dump(k_annos, fp)    

In [50]:
# import csv
# with open(path+"frequencies_20k.csv", "w") as outfile:
#     csvwriter = csv.writer(outfile)
#     csvwriter.writerows(y.items())

In [51]:
anno_data = list(itertools.chain.from_iterable(k_exacts))
anno_count = Counter(anno_data)
anno_y = OrderedDict(anno_count.most_common())

In [52]:
anno_y

OrderedDict([('Cer', 1236),
             ('GFP', 1228),
             ('COI', 614),
             ('ABA', 605),
             ('BPA', 530),
             ('PC1', 508),
             ('PHB', 488),
             ('GST', 442),
             ('Mim', 406),
             ('ATPase', 387),
             ('actin', 384),
             ('MIC', 383),
             ('SRI', 381),
             ('NBS', 374),
             ('std', 364),
             ('PC2', 363),
             ('HGT', 362),
             ('ssp', 335),
             ('FST', 333),
             ('cav', 329),
             ('ESI', 323),
             ('YFP', 323),
             ('ROI', 322),
             ('SPL', 305),
             ('VT1', 303),
             ('ECM', 300),
             ('CA1', 297),
             ('trypsin', 291),
             ('LPS', 282),
             ('IND', 275),
             ('ppm', 273),
             ('TBA', 271),
             ('ANF', 260),
             ('CH2', 254),
             ('AMF', 251),
             ('CNF', 251),
             ('ER

In [53]:
import csv

with open(path+"anno_frequencies_GP_20K.csv", "w",  newline='\n') as outfile:
    public_writer = csv.writer(outfile, delimiter='\t', lineterminator='\n')
    for keys, values in y.items(): # Get keys from the missing
        filt_per = (values/anno_y[keys])*100
        new_row = [keys, str(values), str(anno_y[keys]), str(filt_per)]
        public_writer.writerows([new_row])